# 3. Aligning & Scaling Point Clouds

In order to accurately scale the 3D model point cloud and have it be aligned with the scan cloud, you'll need to align their orientation first. 

## 1. Importing Libraries

In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 2. Importing the files

In [2]:
kinect_scan_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\ImagesAndPointClouds\\pointcloud\\new_pc\\0_color.ply"
model_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\model_point_cloud\\output_10000.ply"
target = o3d.io.read_point_cloud(kinect_scan_path) 
source = o3d.io.read_point_cloud(model_path)

## 3. Centroid to Defining feature line

The following section intends to make use of a line that goes from the centroid of the pepper model to the peduncle in order to use it to align the scan with model. For the kinect scan the peduncle wasn't as easy to reach, so we went with a defining feature, which traversed the scan length wise, giving a sense of its orientation. We use the Z coordinate for the model since the peduncle is located on the highest point of the Z axis. For the scan we use the y-axis due to the way the model is projected into 3D space. 

In [3]:
def compute_centroid(point_cloud): 
    return np.mean(point_cloud.points, axis=0)

In [4]:
def identify_peduncle_point(pcd): 
    z_coordinates = np.asarray(pcd.points)[:,2]

    threshold = np.percentile(z_coordinates, 98)
    top_points = np.asarray(pcd.points)[z_coordinates > threshold]

    return np.mean(top_points, axis=0)

In [5]:
def identify_peduncle_point_scan(pcd): 
    y_coordinates = np.asarray(pcd.points)[:,0]

    threshold = np.percentile(y_coordinates, 98)
    top_points = np.asarray(pcd.points)[y_coordinates > threshold]

    return np.mean(top_points, axis=0)

In [6]:
def compute_line(pcd):
    centroid = compute_centroid(pcd)
    peduncle_point = identify_peduncle_point(pcd)

    direction_vector = peduncle_point - centroid
    normalized_vector = direction_vector / np.linalg.norm(direction_vector)
    return normalized_vector, centroid, peduncle_point

In [7]:
def compute_line_scan(pcd):
    centroid = compute_centroid(pcd)
    peduncle_point = identify_peduncle_point_scan(pcd)

    direction_vector = peduncle_point - centroid
    normalized_vector = direction_vector / np.linalg.norm(direction_vector)
    return normalized_vector, centroid, peduncle_point

In [8]:
direction, centroid, peduncle_point = compute_line(source)

In [10]:
line_points = [centroid, peduncle_point]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(line_points)
line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
o3d.visualization.draw_geometries([source, line_set])

As proven by the visualization, we can see the line from the center of the pepper model all the way up to the peduncle. This gives us a good idea of a rotation axis which we can use to orient it with the model scan more easily. 

In [11]:
direction_scan, centroid_scan, peduncle_point_scan = compute_line_scan(target)

In [13]:
line_points = [centroid_scan, peduncle_point_scan]  
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(line_points)
line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
o3d.visualization.draw_geometries([target, line_set])

For the scan, the line goes to the opposite sense of the peduncle. Still it gives us a good take on the orientation of the pepper scan. 

The origin of the scanned point cloud is not in the pepper. It is in one of the corners of the image. In order to make working with the scan easier, we moved the origin to the centroid of the pepper. 

In [14]:
import open3d as o3d
import numpy as np

# Load original point cloud
pcd = o3d.io.read_point_cloud(kinect_scan_path)

# Compute centroid
centroid = np.mean(np.asarray(pcd.points), axis=0)

# Create point cloud for the centroid
centroid_pcd = o3d.geometry.PointCloud()
centroid_pcd.points = o3d.utility.Vector3dVector([centroid])

# Assign red color to the centroid
centroid_color = [1, 0, 0]  # RGB for red
centroid_pcd.colors = o3d.utility.Vector3dVector([centroid_color])

# Create a coordinate frame (axes) at the origin (can be adjusted to another position)
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# Visualize everything together
o3d.visualization.draw_geometries([pcd, centroid_pcd, coord_frame])

In [15]:
import open3d as o3d
import numpy as np

# Load original point cloud
pcd = o3d.io.read_point_cloud(kinect_scan_path)

# Compute centroid
centroid = np.mean(np.asarray(pcd.points), axis=0)

# Translate point cloud to make the centroid the origin
translated_points = np.asarray(pcd.points) - centroid
pcd.points = o3d.utility.Vector3dVector(translated_points)

# Create point cloud for the centroid
centroid_pcd = o3d.geometry.PointCloud()
centroid_pcd.points = o3d.utility.Vector3dVector([centroid])

# Assign red color to the centroid
centroid_color = [1, 0, 0]  # RGB for red
centroid_pcd.colors = o3d.utility.Vector3dVector([centroid_color])

# Create a coordinate frame (axes) at the origin (can be adjusted to another position)
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# Save and/or visualize the translated point cloud
o3d.io.write_point_cloud("C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\translated_kinect_scan\\translated_kinect_scan.ply", pcd)
o3d.visualization.draw_geometries([pcd, centroid_pcd, coord_frame])


After having moved the origin to the centroid, we can now start the alignment operations. We compute the center line again first, just to make sure we're doing the alignment right. 

In [16]:
translated_kinect_scan_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\translated_kinect_scan\\translated_kinect_scan.ply"
translated_kinect_scan = o3d.io.read_point_cloud(translated_kinect_scan_path)
direction_scan, centroid_scan, peduncle_point_scan = compute_line_scan(translated_kinect_scan)
line_points = [centroid_scan, peduncle_point_scan]  
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(line_points)
line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
o3d.visualization.draw_geometries([translated_kinect_scan, line_set])

## 4. Aligning the models based on their center lines. 

In [17]:
def compute_rotation(v1, v2):
    # Normalize vectors
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    
    # Compute rotation axis
    rotation_axis = np.cross(v1, v2)
    rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
    
    # Compute rotation angle
    cos_angle = np.dot(v1, v2)
    rotation_angle = np.arccos(cos_angle)
    
    return rotation_axis, rotation_angle

In [18]:
def axis_angle_to_rotation_matrix(axis, angle):
    # Using the Rodrigues' rotation formula
    K = np.array([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ])
    I = np.eye(3)
    
    R = I + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)
    return R

In [19]:
def apply_rotation(pcd, rotation_matrix):
    return np.dot(pcd, rotation_matrix.T)  # .T is for transpose

In [20]:
initial_direction = direction / np.linalg.norm(direction)
target_direction = direction_scan / np.linalg.norm(direction_scan)

In [21]:
# Compute the rotation
axis, angle = compute_rotation(initial_direction, target_direction)
R = axis_angle_to_rotation_matrix(axis, angle)

source_points = np.asarray(source.points)

# Apply rotation to 3D model
rotated_points_np = apply_rotation(source_points, R)

In [22]:
rotated_pcd = o3d.geometry.PointCloud()
rotated_pcd.points = o3d.utility.Vector3dVector(rotated_points_np)

In [23]:
o3d.visualization.draw_geometries([rotated_pcd, translated_kinect_scan])

After having performed the initial alignment, we can see that the peduncles are in opposite sides of the orientations. So... We need to rotate one of the two in the Z-axis. 

In [24]:
def rotation_matrix_around_y(angle_rad):
    return np.array([
        [np.cos(angle_rad), 0, np.sin(angle_rad)],
        [0, 1, 0],
        [-np.sin(angle_rad), 0, np.cos(angle_rad)]
    ])

In [25]:
def rotation_matrix_around_x(angle_rad):
    return np.array([
        [1, 0, 0],
        [0, np.cos(angle_rad), -np.sin(angle_rad)],
        [0, np.sin(angle_rad), np.cos(angle_rad)]
    ])

In [26]:
def rotation_matrix_around_z(angle_rad):
    return np.array([
        [np.cos(angle_rad), -np.sin(angle_rad), 0],
        [np.sin(angle_rad), np.cos(angle_rad), 0],
        [0, 0, 1]
    ])

In [27]:
angle_180_degrees = (np.pi)  # 180° in radians
R_180 = rotation_matrix_around_z(angle_180_degrees)
flipped_points_np = apply_rotation(rotated_points_np, R_180)

In [28]:
flipped_pcd = o3d.geometry.PointCloud()
flipped_pcd.points = o3d.utility.Vector3dVector(flipped_points_np)

In [29]:
o3d.visualization.draw_geometries([flipped_pcd, target])

In [30]:
starting_alignment_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\starting_alignment_model\\starting_alignment_model.ply"
o3d.io.write_point_cloud(starting_alignment_path, flipped_pcd)

True

## 5. Scaling the model

Once the desired orientation is achieved, now we can scan our model based on the bounding boxes of the kinect scan. 

This is done to make the model be as close in dimensions to the kinect scan as possible. 

In [31]:
# The name of the flipped model is "flipped_pcd"
def get_dimensions(pcd):
    bounding_box = pcd.get_axis_aligned_bounding_box()
    return bounding_box.get_extent()


def scale_point_cloud(source_pcd, target_dimensions, source_dimensions=None):
    if source_dimensions is None:
        source_dimensions = get_dimensions(source_pcd)
    
    scale_factors = [
        target_dimensions[i] / source_dimensions[i]
        for i in range(3)
    ]

    scaled_points = [
        [scale_factors[j] * pt[j] for j in range(3)]
        for pt in source_pcd.points
    ]

    scaled_pcd = o3d.geometry.PointCloud()
    scaled_pcd.points = o3d.utility.Vector3dVector(scaled_points)
    return scaled_pcd

In [32]:
target_dimensions = get_dimensions(translated_kinect_scan)
scaled_pcd2 = scale_point_cloud(flipped_pcd, target_dimensions)

In [33]:
# We paint the point clouds to be able to distinguish between them
translated_kinect_scan.paint_uniform_color([1, 0, 0])  # Paint the first point cloud red
scaled_pcd2.paint_uniform_color([0, 1, 0])  # Paint the scaled point cloud green

PointCloud with 10000 points.

In [34]:
o3d.visualization.draw_geometries([translated_kinect_scan, scaled_pcd2])

In [35]:
scaled_aligned_model_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\scaled_aligned_model\\pancake_right_inclination.ply"
o3d.io.write_point_cloud(scaled_aligned_model_path, scaled_pcd2)

True

The models have now been scaled. 